In [24]:
import pandas as pd
import torch.nn as nn
import torch
import torch.optim as optim
from tqdm.notebook import tqdm
from torch.utils.data import DataLoader, TensorDataset, random_split
import torch.nn.functional as F
from tqdm.notebook import tqdm
import time
import copy

In [2]:
#3 layer MLP with one hidden layer
class MultilayerPerceptron(nn.Module):
    #define network
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()
        self.fc1 = nn.Linear(input_size, hidden_size) # input layer -> hidden layer
        self.tanh = nn.Tanh()
        self.fc2 = nn.Linear(hidden_size, output_size)

    #define forward function
    def forward(self, x):
        out = self.fc1(x)
        out = self.tanh(out)
        out = self.fc2(out)
        return out
    
    

In [12]:
def training(model, dataloader, func_loss, opt, num_epochs, experiment_name):
    model.train()
    best_model = None
    best_acc = 0.0
    
    for epoch in range(num_epochs):
        running_loss = 0.0
        running_corrects = 0
        for i, batched_data in enumerate(dataloader): 
            inputs, targets = batched_data
            print("inputs: " + str(inputs))
            print("target: " + str(targets))
            break
            
            output_prob = model(inputs) #pass images through model to get probability for each class (1 x 10 dim)
            loss = func_loss(output_prob.squeeze(), targets)
            
            # Backward and optimize
            loss.backward() #performing back prop
            opt.step() #updating the weights
            opt.zero_grad() #reset gradients
            
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(output_prob == targets.data)
        
        epoch_loss = running_loss / 100
        epoch_acc = running_corrects.double() / 100
        
        if epoch_acc > best_acc:
            best_acc = epoch_acc
            best_model = copy.deepcopy(model.state_dict())
            torch.save(best_model, os.path.join(experiment_name,
                                                'best_model.pt'))
            
        if epoch % 5 == 0:
            if epoch_acc > 50:
                print("accuracy for epoch: " + str(epoch_acc))
            print('Epoch: ' + str(epoch) + ' Final loss is: ' + str(loss.item()))
            
    print('Epoch: ' + str(epoch) + ' Final loss is: ' + str(loss.item()))

In [19]:
def train_model(model, dataloaders, criterion, optimizer, num_epochs=25):
    since = time.time()

    val_acc_history = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to('cuda:0')
                labels = labels.to('cuda:0')

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                
                    # print("OUTPUTS")
                    outputs = model(inputs)
                    # print(outputs)
                    loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)
                    # print("PREDS")
                    # print(preds)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_acc)

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, val_acc_history

In [18]:

df = pd.read_csv('../Data/PGH/DemandPrediction/learning_set_unnormalized.csv')
df = df.drop(columns=['Unnamed: 0', 'index', 'month', 'year'])
df = df[df['outflow']>0]
            

X = torch.from_numpy(df.iloc[:, 0:4].to_numpy()).type(torch.float)
y = torch.from_numpy(df.iloc[:, 4].to_numpy()).type(torch.float)

# mean = torch.mean(X, axis=0)
# stdev = torch.std(X, axis=0)
# X = (X - mean)/stdev
print(len(X))

dataset = TensorDataset(X, y)

train_ds, val_ds = random_split(dataset, [1300, 626])
batch_size=10
train_loader = DataLoader(train_ds, batch_size, shuffle=True)
val_loader = DataLoader(val_ds, batch_size, shuffle=True)
dataloaders = {'train': train_loader, 'val': val_loader}

1926


In [20]:

input_size = len(df.iloc[:,0:4].columns)
output_size = 1
input_size,output_size

(4, 1)

In [25]:
model = nn.Sequential(nn.Linear(input_size, 5), nn.ELU(),
                     nn.Linear(5, 9), nn.ELU(),
                     nn.Linear(9, 9), nn.ELU(),
                     nn.Linear(9, 5), nn.ELU(),
                     nn.Linear(5, 1))
epochs = 500
learning_rate = 1e-3
optimizer = optim.SGD(model.parameters(), lr=learning_rate, weight_decay=1e-4, momentum=0.99)
model_tr, val_hist = train_model(model, dataloaders, nn.L1Loss(), optimizer, epochs)

Epoch 0/499
----------


AssertionError: Torch not compiled with CUDA enabled

In [15]:
def evaluation(model, dataloader):
    model.eval()
    total_loss = 0
    
    with torch.no_grad():
        right, running_total = 0, 0

        for i, batched_data in enumerate(dataloader):

            #moving tensors to the GPU
            inputs, targets = batched_data #grab the batch of images as tensors and grab the batch of labels as tensors

            output_probs = model(inputs) #pass images through model to get probability for each class (1 x 10 dim)
            _, pred_label = torch.max(output_probs.data, 1)
            
            running_total += targets.size(0)
            right += (pred_label == targets).sum().item()
            
    acc = (right / running_total)
    print("Accuracy is " + str(acc))

In [16]:
# model = MultilayerPerceptron(input_size, 150, output_size)
model = nn.Sequential(nn.Linear(input_size, 5), nn.ELU(),
                     nn.Linear(5, 9), nn.ELU(),
                     nn.Linear(9, 9), nn.ELU(),
                     nn.Linear(9, 5), nn.ELU(),
                     nn.Linear(5, 1))
epochs = 500
learning_rate = 1e-3
optimizer = optim.RMSprop(model.parameters(), lr=learning_rate, weight_decay=1e-4, momentum=0.99)
training(model, train_loader, nn.L1Loss(), optimizer, epochs, "test1")

inputs: tensor([[97.0000, 81.0000, 97.0000, 70.8000],
        [78.0000, 81.0000, 65.0000, 70.8000],
        [76.0000, 96.0000, 49.0000, 69.7000],
        [97.0000, 93.0000, 71.0000, 73.0000],
        [79.0000, 78.0000, 44.0000, 34.9000],
        [84.0000, 72.0000, 64.0000, 71.9000],
        [78.0000, 81.0000, 65.0000, 69.6000],
        [73.0000, 66.0000, 71.0000, 52.3000],
        [89.0000, 82.0000, 60.0000, 37.3000],
        [83.0000, 89.0000, 71.0000, 69.5000]])
target: tensor([545., 401.,  50., 138.,  19.,  54., 603.,   7.,  19., 181.])


UnboundLocalError: local variable 'loss' referenced before assignment

In [11]:
evaluation(model, test_loader)

Accuracy is 0.0
